In [1]:
import numpy as np
import pandas as pd
import psycopg2

In [2]:
import json
from pprint import pprint

In [7]:
db = psycopg2.connect("dbname=movies  user=postgres password=littlefella")
cursor = db.cursor()
cursor

<cursor object at 0x110e04ed0; closed: 0>

In [8]:
cursor.execute("SELECT COUNT(*) FROM ml_movies;")

In [9]:
cursor.rowcount

1

In [10]:
cursor.fetchall()

[(62423,)]

In [48]:
movies = pd.read_csv(
    "data/movie_lens/movies.csv",
    dtype=str
)
movies.genres = movies.genres.apply(lambda g: g.lower().split("|"))
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[adventure, animation, children, comedy, fantasy]"
1,2,Jumanji (1995),"[adventure, children, fantasy]"
2,3,Grumpier Old Men (1995),"[comedy, romance]"
3,4,Waiting to Exhale (1995),"[comedy, drama, romance]"
4,5,Father of the Bride Part II (1995),[comedy]


In [49]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
movieId    62423 non-null object
title      62423 non-null object
genres     62423 non-null object
dtypes: object(3)
memory usage: 1.4+ MB


In [51]:
import re

In [56]:
re.sub(r"\s*\((\d+)\)","","Toy Story (1995)").strip()

'Toy Story'

In [148]:
db.rollback()

In [114]:
cursor.execute("INSERT INTO ml_movies (movieId, title, year) VALUES (%s, %s, %s);", ("1", "Toy Story", 1995))
cursor.rowcount

1

In [115]:
db.commit()

In [154]:
cursor.execute("SELECT * FROM ml_movies;")
cursor.fetchall()

OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.


In [153]:
db.status

1

In [149]:
cursor.execute("DELETE FROM ml_genres;")
cursor.execute("DELETE FROM ml_tags;")
cursor.execute("DELETE FROM ml_movies;")
print(cursor.rowcount)
db.commit()
cursor.execute("SELECT * FROM ml_movies;")
cursor.fetchall()

KeyboardInterrupt: 

In [4]:
from tqdm import tqdm

In [159]:
for mid, title, genres in tqdm(movies.values):
    try:
        year = int(re.search(r"\((\d+)\)\s*$",title)[1])
    except Exception as e:
#         print(e)
        year = None
    try:
        t = re.sub(r"\s*\((\d+)\)\s*$","",title).strip()
    except Exception as e:
#         print(e)
        t = title
    try:
        cursor.execute("INSERT INTO ml_movies (movieId, title, year) VALUES (%s, %s, %s);",(mid, t, year))
        for g in genres:
            cursor.execute("INSERT INTO ml_genres (movieId, genre) VALUES (%s, %s);",(mid, g))
    except Exception as e:
#         print("Error! Rolling back and then raising...")
        db.rollback()
        raise e
    else:
        db.commit()


100%|██████████| 62423/62423 [00:33<00:00, 1889.87it/s]

In [70]:
tags_df = pd.read_csv("data/movie_lens/tags.csv")
tags_df.userId = tags_df.userId.astype(str)
tags_df.movieId = tags_df.movieId.astype(str)
tags_df.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [71]:
tags_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093360 entries, 0 to 1093359
Data columns (total 4 columns):
userId       1093360 non-null object
movieId      1093360 non-null object
tag          1093344 non-null object
timestamp    1093360 non-null int64
dtypes: int64(1), object(3)
memory usage: 33.4+ MB


In [ ]:
tags_df.tag.value_counts().reset_index().head()

In [141]:
tags_df.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [146]:
db.rollback()

 33%|███▎      | 360623/1093360 [02:20<04:03, 3013.79it/s]

In [160]:
uids = tags_df.userId.unique()
for uid in tqdm(uids):
    try:
        cursor.execute("INSERT INTO ml_users ( userId ) VALUES ( %s );",(uid,))
    except Exception as e:
        db.rollback()
        raise e
    else:
        db.commit()


100%|██████████| 14592/14592 [00:04<00:00, 3123.32it/s]

In [161]:
for t in tqdm(tags_df.values):
    try:
#         cursor.execute("SELECT COUNT(*) FROM ml_users WHERE userId = %s;",(uid,))
#         if cursor.fetchone()[0] > 0:
#             cursor.execute("INSERT INTO ml_users (userId) VALUES %s;",(uid,))
        cursor.execute(
            "INSERT INTO ml_tags (userId, movieId, tag, timestamp) VALUES (%s, %s, %s, %s)",
            t
        )
    except Exception as e:
        db.rollback()
        raise e
    else:
        db.commit()


100%|██████████| 1093360/1093360 [06:29<00:00, 2807.70it/s]

In [88]:
!head data/movie_lens/ratings.csv

userId,movieId,rating,timestamp
1,296,5.0,1147880044
1,306,3.5,1147868817
1,307,5.0,1147868828
1,665,5.0,1147878820
1,899,3.5,1147868510
1,1088,4.0,1147868495
1,1175,3.5,1147868826
1,1217,3.5,1147878326
1,1237,5.0,1147868839


In [90]:
!head data/movie_lens/genome-scores.csv

movieId,tagId,relevance
1,1,0.028749999999999998
1,2,0.023749999999999993
1,3,0.0625
1,4,0.07574999999999998
1,5,0.14075
1,6,0.14675
1,7,0.0635
1,8,0.20375
1,9,0.202


In [91]:
!head data/movie_lens/genome-tags.csv

tagId,tag
1,007
2,007 (series)
3,18th century
4,1920s
5,1930s
6,1950s
7,1960s
8,1970s
9,1980s


In [92]:
!wc -l data/movie_lens/ratings.csv

 25000096 data/movie_lens/ratings.csv


In [102]:
chunksize = 100_000
print("Number of iterations (using chunksize: %d): %s" % (chunksize, 25_000_096 / chunksize))

Number of iterations (using chunksize: 100000): 250.00096


In [106]:
ratings_df = pd.read_csv("data/movie_lens/ratings.csv",nrows=1000,dtype={"userId":str,"movieId":str,"rating":float,"timestamp":int})
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [104]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
userId       1000 non-null int64
movieId      1000 non-null int64
rating       1000 non-null float64
timestamp    1000 non-null int64
dtypes: float64(1), int64(3)
memory usage: 31.3 KB


In [177]:
cursor.execute("DELETE FROM ml_ratings;")
print(cursor.rowcount)
db.commit()

0


In [175]:
db.rollback()

In [5]:
for ratings_df in tqdm(pd.read_csv(
    "data/movie_lens/ratings.csv",
    chunksize=10_000,
    dtype={"userId":str,"movieId":str,"rating":float,"timestamp":int}
)):
    for uid in ratings_df.userId.unique():
        try:
            cursor.execute("INSERT INTO ml_users (userId) VALUES (%s);", (uid,))
        except:
            db.rollback()
        else:
            db.commit()
    for t in (ratings_df.values):
        try:
            cursor.execute("INSERT INTO ml_ratings (userId, movieId, rating, timestamp) VALUES (%s,%s,%s,%s);",t)
        except Exception as e:
            db.rollback()
            raise e
        else:
            db.commit()

2501it [2:06:30,  3.04s/it]


In [11]:
cursor.execute("SELECT COUNT(*) FROM ml_ratings;")
print(cursor.fetchone())
print(cursor.rowcount)
db.commit()

OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
